# Intro
The following code can be used to generate labels for screw terminals

## Program Dependencies

In [1]:
from PIL import Image, ImageDraw, ImageFont
import os

pixels_per_inch = 300
inch_to_mm = 25.4

def mm_to_pixels(mm):
    return int((mm / inch_to_mm) * pixels_per_inch)

def pixels_to_mm(pixels):
    return (pixels / pixels_per_inch) * inch_to_mm

def textsize(text, font):
    im = Image.new(mode="P", size=(0, 0))
    draw = ImageDraw.Draw(im)
    _, _, width, height = draw.textbbox((0, 0), text=text, font=font)
    return width, height

class ScrewTerminalLabel:
    """Class to represent a single screw terminal.

    @param text: The text for the terminal.
    @param width: Width of the terminal in mm.
    @param height: Height of the terminal in mm.
    @param text_color: The color of the terminal (optional).
    """

    def __init__(self, pin, text, width, height, background_color="white", text_color="black"):
        self.pin = pin
        self.text = text
        self.width = width
        self.height = height
        self.background_color = background_color  # Store the background color of the label
        self.text_color = text_color

    def generate_label_image(self):
        """Generate an image for the terminal label."""
        width_pixels = mm_to_pixels(self.width)
        height_pixels = mm_to_pixels(self.height)

        # Create a blank image with the background color
        image = Image.new("RGB", (width_pixels, height_pixels), self.background_color)
        draw = ImageDraw.Draw(image)

        # Load a font
        try:
            if os.name == "posix": # macOS or Linux
                font_path = "/System/Library/Fonts/Supplemental/DIN Condensed Bold.ttf"
            elif os.name == "nt": # Windows
                font_path = "C:\\Windows\\Fonts\\bahnschrift.ttf"
                font.set_variation_by_name("Condensed")
            font = ImageFont.truetype(font_path, size=int(height_pixels * 0.75))
        except IOError:
            font = ImageFont.load_default()

        # Calculate text size and position
        text_width, text_height = textsize(self.text, font=font)
        text_x = (width_pixels - text_width) / 2
        text_y = (height_pixels - text_height) / 2

        # Draw the text on the image
        draw.text((text_x, text_y), self.text, fill=self.text_color, font=font)

        return image

class ScrewTerminalBlock:
    """Class to represent a screw terminal block.

    @param number_of_terminals: Number of terminals in the block.
    @param start_pin: The pin number of the first terminal.
    @param end_pin: The pin number of the last terminal.
    @param length: Length of the terminal block label section in mm.
    @param height: Height of the terminal block label section in mm.

    """
    def __init__(self, number_of_terminals, start_pin, end_pin, length, height):
        self.number_of_terminals = number_of_terminals
        self.start_pin = start_pin
        self.end_pin = end_pin
        self.length = length
        self.width = height
        self.terminal_label = [ScrewTerminalLabel] * number_of_terminals

        for i in range(number_of_terminals):
            pin = start_pin + i
            label = f"{pin}"
            self.terminal_label[i] = ScrewTerminalLabel(pin, label, length / number_of_terminals, height)

    @property
    def pin_range(self):
        """Get the range of pin numbers for the terminals."""
        return range(self.start_pin, self.end_pin + 1)
    
    def generate_block_image(self):
        """Generate an image for the terminal block."""
        total_width_pixels = mm_to_pixels(self.length)
        height_pixels = mm_to_pixels(self.width)

        # Create a blank image for the terminal block
        block_image = Image.new("RGB", (total_width_pixels, height_pixels), "white")
        
        # Calculate width of each terminal label
        terminal_width_pixels = total_width_pixels / self.number_of_terminals
        
        for i, terminal in enumerate(self.terminal_label):
            terminal_image = terminal.generate_label_image()
            # Calculate position to paste the terminal label
            x_position = int(i * terminal_width_pixels)
            block_image.paste(terminal_image, (x_position, 0))
            
            # Add a division line 
            if i < self.number_of_terminals - 1:
                draw = ImageDraw.Draw(block_image)
                divider_width = 3
                draw.line((x_position + terminal_width_pixels - divider_width / 2, 0, x_position + terminal_width_pixels - divider_width / 2, height_pixels), fill="black", width=divider_width)

        return block_image
    



## Program Input

In [2]:
terminal_block = ScrewTerminalBlock(5, 1, 5, 19, 3.5)

terminal_block.terminal_label[0].text = "1"
terminal_block.terminal_label[1].text = "2"
terminal_block.terminal_label[2].text = "IN"
terminal_block.terminal_label[3].text = "RX"
terminal_block.terminal_label[3].background_color = "yellow"
terminal_block.terminal_label[4].text = "5V"
terminal_block.terminal_label[4].background_color = "red"

block_image = terminal_block.generate_block_image()
block_image.show()
block_image.save("terminal_block_1.png")

terminal_block = ScrewTerminalBlock(5, 5, 9, 19, 3.5)

terminal_block.terminal_label[0].text = "GND"
terminal_block.terminal_label[0].background_color = "black"
terminal_block.terminal_label[0].text_color = "white"
terminal_block.terminal_label[1].text = "SWD"
terminal_block.terminal_label[1].background_color = "yellow"
terminal_block.terminal_label[2].text = "CLK"
terminal_block.terminal_label[2].background_color = "green"
terminal_block.terminal_label[3].text = "OUT"
terminal_block.terminal_label[4].text = "TX"
terminal_block.terminal_label[4].background_color = "green"


block_image = terminal_block.generate_block_image()
block_image.show()
block_image.save("terminal_block_2.png")


## Program Processing

## Program Output